# <font class='ign_color'>TF in ROS</font>

## Chapter 2: TF Publish and Subscribe

## 1. TF Publisher

Now it's your turn to create your own TF publisher. These publishers are called <b>TF broadcasters</b>.

The following code publishes the TFs for the turtle (turtle1) and the iRobot (turtle2).<br>
Please have a look at the below code first and try to guess what each piece of it does.<br>
Then, continue reading for a detailed explanation of the main elements of the code.

<p style="background:#3B8F10;color:white;" id="unit2_example1">
Python Program {U2.1.1.py}: multiple_broadcatser.py
</p>

In [1]:
#! /usr/bin/env python
import rospy
import time
import tf
from turtle_tf_3d.get_model_gazebo_pose import GazeboModel


def handle_turtle_pose(pose_msg, robot_name):
    br = tf.TransformBroadcaster()
    
    br.sendTransform((pose_msg.position.x,pose_msg.position.y,pose_msg.position.z),
                     (pose_msg.orientation.x,pose_msg.orientation.y,pose_msg.orientation.z,pose_msg.orientation.w),
                     rospy.Time.now(),
                     robot_name,
                     "/world")

def publisher_of_tf():
    
    rospy.init_node('publisher_of_tf_node', anonymous=True)
    robot_name_list = ["turtle1","turtle2"]
    gazebo_model_object = GazeboModel(robot_name_list)
    
    
    for robot_name in robot_name_list:
        pose_now = gazebo_model_object.get_model_pose(robot_name)
    
    # Leave enough time to be sure the Gazebo Model logs have finished
    time.sleep(1)
    rospy.loginfo("Ready..Starting to Publish TF data now...")
    
    rate = rospy.Rate(5) # 5hz
    while not rospy.is_shutdown():
        for robot_name in robot_name_list:
            pose_now = gazebo_model_object.get_model_pose(robot_name)
            if not pose_now:
                print "The Pose is not yet"+str(robot_name)+" available...Please try again later"
            else:
                handle_turtle_pose(pose_now, robot_name)
        rate.sleep()
    

if __name__ == '__main__':
    try:
        publisher_of_tf()
    except rospy.ROSInterruptException:
        pass

SyntaxError: invalid syntax (<ipython-input-1-34cf0dd3d72b>, line 36)

<p style="background:#3B8F10;color:white;" id="unit2_example1">
END Python Program {U2.1.1.py}: multiple_broadcatser.py
</p>

### How to get the robot's position?

This will depend on the localization system of your robot.<br>
It could be retrieved from the /odom topic publishing the <b>odometry</b>.<br>
It could be obtained from a <b>navigation system</b>.<br>
It could be calculated through <b>GPS data</b>.<br>

In this case, because it's a simulated environment, the Gazebo Simulator used can give you the position and orientation where each model is with perfect precision.<br>
You can use a class that we created for making your life easier, called <b>GazeboModel</b>.<br>
This class, among other functionalities, gives you the current pose of a model each time you ask for it.
You just have to call the function <b>get_model_pose(robot_name)</b>.
You can import it from the <b>turtle_tf_3d</b> package, in the <b>get_model_gazebo_pose</b> module.

In [ ]:
from turtle_tf_3d.get_model_gazebo_pose import GazeboModel
#
#
#
robot_name_list = ["turtle1","turtle2"]
gazebo_model_object = GazeboModel(robot_name_list)
    
for robot_name in robot_name_list:
    pose_now = gazebo_model_object.get_model_pose(robot_name)

These lines of code make it possible to get the position and orientation of a Gazebo Model if it's in the current simulation.

### How do you transmit the current pose to the broadcaster?

In [ ]:
rate = rospy.Rate(5) # 5hz
while not rospy.is_shutdown():
    for robot_name in robot_name_list:
        pose_now = gazebo_model_object.get_model_pose(robot_name)
        if not pose_now:
            print "The Pose is not yet"+str(robot_name)+" available...Please try again later"
        else:
            handle_turtle_pose(pose_now, robot_name)
    rate.sleep()

Essentially, this piece of code just gets the current pose of the desired models and sends it to the TF publisher, at a certain rate. 5 Hz, in this case.

### How the TF is published?

In [ ]:
def handle_turtle_pose(pose_msg, robot_name):
    br = tf.TransformBroadcaster()
    
    br.sendTransform((pose_msg.position.x,pose_msg.position.y,pose_msg.position.z),
                     (pose_msg.orientation.x,pose_msg.orientation.y,pose_msg.orientation.z,pose_msg.orientation.w),
                     rospy.Time.now(),
                     robot_name,
                     "/world")

This function is the one that makes the TF publish.<br>
<ul>
    <li>It creates an instance of the TransformBroadCaster</li>
</ul>

In [ ]:
br = tf.TransformBroadcaster()

<ul>
    <li>
    Publish the transform of the pose message:<br>
    You have to publish each element of the position and orientation inside a parenthesis, otherwise it might not work.<br>
    There is also a very important element, which is the <b>rospy.Time.now()</b>. This is because TF really depends on time to make everything work and be able to play with past messages.<br>
    Then, state the name of the child-frame you want to assign that model (robot_name) and the parent-frame, which, in this case, is <b>/world</b>
    </li>
</ul>

In [ ]:
br.sendTransform((pose_msg.position.x,pose_msg.position.y,pose_msg.position.z),
                     (pose_msg.orientation.x,pose_msg.orientation.y,pose_msg.orientation.z,pose_msg.orientation.w),
                     rospy.Time.now(),
                     robot_name,
                     "/world")

<p style="background:#EE9023;color:white;">Exercise 2.1</p>
<br>
In this exercise, you are going to create your first TF publisher to publish the position and orientation of both turtle1 and turtle2.

1.- First step: Stop any launch you had from previous exercises. For example, the <b>roslaunch turtle_tf_3d irobot_follow_turtle.launch</b>.

2.- Create a new ROS package that has <b>rospy and turtle_tf_3d</b> as dependencies.<br>
3.- Create a python script that contains an explanation of the code above <a href="#unit2_example1">(Python Program {U2.1.1.py})</a>.

4.- Execute your python code:
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rosrun your_package your_tf_broadcaster.py

5.-Now, launch the following, which activates the TF topic listener in another WebShell. Then, based on the TF of both turtle1 and turtle2, this listener will calculate the direction and speed necessary to make the iRobot follow the turtle.
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
roslaunch turtle_tf_3d run_turtle_tf_listener.launch

6.- Check that everything is working as it should with the tf_echo, rqt_tf_tree, and RVIZ tools

<p style="background:#EE9023;color:white;">END Exercise 2.1</p>

If all went well, the iRobot should be trying to follow the turtle now.

<p style="background:#EE9023;color:white;">Exercise 2.2</p>
<br>
In this exercise, you are going to create a new publisher that publishes the TF of a new model, the coke_can.
Here is data to help you on this exercise:
<ol>
    <li>
    Execute the following command in the WebShell to see the exact name of the coke can model:<br><br>
    **<i>rosservice call /gazebo/get_world_properties "{}"</i>**
    </li>
    <br>
    <li>You have to retrieve its position in the same way that you did with the turtle and iRobot</li>
<ol>

<p style="background:#EE9023;color:white;">END Exercise 2.2</p>

If all went well, you should now have a third TF published for the coke_can, similar to the following:

<img src="img/coke_can_exercise.png"/>

<img src="img/coke_can_exercise_rviz.png"/>

## 2. Subscriber

Now it is time to do something useful with that published TF data. It's time you create a <b>listener</b>.

<p style="background:#3B8F10;color:white;" id="unit2_example2">
Python Program {U2.2.1.py}: model1_to_model2_listener.py
</p>

In [ ]:
#!/usr/bin/env python
import sys
import rospy
import math
import tf
import geometry_msgs.msg

if __name__ == '__main__':
    rospy.init_node('tf_listener_turtle')

    listener = tf.TransformListener()

    if len(sys.argv) < 3:
        print("usage: turtle_tf_listener.py follower_model_name model_to_be_followed_name")
    else:
        follower_model_name = sys.argv[1]
        model_to_be_followed_name = sys.argv[2]
        
        turtle_vel = rospy.Publisher(follower_model_name+'/cmd_vel', geometry_msgs.msg.Twist,queue_size=1)
    
        rate = rospy.Rate(10.0)
        ctrl_c = False
        
        follower_model_frame = "/"+follower_model_name
        model_to_be_followed_frame = "/"+model_to_be_followed_name
        
        def shutdownhook():
            # works better than the rospy.is_shut_down()
            global ctrl_c
            print "shutdown time! Stop the robot"
            cmd = geometry_msgs.msg.Twist()
            cmd.linear.x = 0.0
            cmd.angular.z = 0.0
            turtle_vel.publish(cmd)
            ctrl_c = True

        rospy.on_shutdown(shutdownhook)
        
        while not ctrl_c:
            try:
                (trans,rot) = listener.lookupTransform(follower_model_frame, model_to_be_followed_frame, rospy.Time(0))
            except (tf.LookupException, tf.ConnectivityException, tf.ExtrapolationException):
                continue
    
            angular = 4 * math.atan2(trans[1], trans[0])
            linear = 0.5 * math.sqrt(trans[0] ** 2 + trans[1] ** 2)
            cmd = geometry_msgs.msg.Twist()
            cmd.linear.x = linear
            cmd.angular.z = angular
            turtle_vel.publish(cmd)
    
            rate.sleep()

<p style="background:#3B8F10;color:white;" id="unit2_example2">
END Python Program {U2.2.1.py}: model1_to_model2_listener.py
</p>

### How to move the iRobot

The movement of each robot will be different, of course. In this particular case, you have to publish the following in the topic:

In [ ]:
#
#
#
turtle_vel = rospy.Publisher(follower_model_name+'/cmd_vel', geometry_msgs.msg.Twist,queue_size=1)
#
#
#
angular = 4 * math.atan2(trans[1], trans[0])
linear = 0.5 * math.sqrt(trans[0] ** 2 + trans[1] ** 2)
cmd = geometry_msgs.msg.Twist()
cmd.linear.x = linear
cmd.angular.z = angular
turtle_vel.publish(cmd)

What is interesting is how it's calculated the Twist because it's extracted from the TF data.

In [ ]:
(trans,rot) = listener.lookupTransform(follower_model_frame, model_to_be_followed_frame, rospy.Time(0))

Remember when you executed the command:

You got the output:

In [ ]:
- Translation: [0.654, 0.018, 0.040]
- Rotation: in Quaternion [-0.000, -0.000, 1.000, -0.013]
            in RPY (radian) [-0.000, 0.000, -3.115]
            in RPY (degree) [-0.000, 0.019, -178.502]

You are using the translation x and y components to calculate the angle difference between the frame of turtle1 and turtle2 on the X-Y plane. Based on that difference, the angular speed will be larger with larger differences and, therefore, turn faster.
The same is true for the linear speed. The linear speed depends on the distance on the X-Y plane between the turtle1 frame and the turtle2 frame. The greater the difference, the more speed for iRobot.

<figure>
    <img id="fig-A.2" src="img/axis_2d.png" width="600"/>
    <center>
        <figcaption><h2>Abstract representation. Bear in mind that it's not the exact transformation, but its merely for helping understand the code that was just explained.</h2></figcaption>
    </center>
</figure>

<p style="background:#EE9023;color:white;">Exercise 2.3</p>
<br>
In this exercise, you are going to create your first TF listener. You should listen to the TF published and move the <b>iRobot</b> to follow:
<ol>
    <li>/turtle1</li>
    <li>/coke_can</li>
<ol>

<p style="background:#EE9023;color:white;">END Exercise 2.3</p>

You should see how the iRobot goes to the position of the realturtle or the coke_can.

## 3. Add More Frames

Ok, now, what if you wanted to add extra frames referred to the /turtle1 frame? For example, a frame for each eye; for the iRobot, a frame for the ir-sensor.<br>
Or even, what if you want to generate a totally abstract frame that turns the iRobot around in circles?<br>
Well, let's find out!

Here you have two examples of publishing frames: a fixed frame (that doesn't change over time) and a moving frame.<br>
You could base their position on sensor readings, simulation parameters, or any other data. In this case, they will be set hardcoded.<br>
We will call these new frames <b>/fixed_carrot</b> and <b>/moving_carrot</b>.

<p style="background:#3B8F10;color:white;" id="unit2_example3">
Python Program {U2.3.1.py}: fixed_extra_frame_pub.py
</p>

In [ ]:
#!/usr/bin/env python  
import rospy
import tf

if __name__ == '__main__':
    rospy.init_node('my_fixed_carrot_tf_broadcaster')
    br = tf.TransformBroadcaster()
    rate = rospy.Rate(3.0)
    while not rospy.is_shutdown():
        br.sendTransform((0.0, 2.0, 0.0),
                         (0.0, 0.0, 0.0, 1.0),
                         rospy.Time.now(),
                         "fixed_carrot",
                         "turtle1")
        rate.sleep()

<p style="background:#3B8F10;color:white;">
END Python Program {U2.3.1.py}: fixed_extra_frame_pub.py
</p>

The broadcaster publishes this new /fixed_carrot frame referred to the /turtle1 frame. It has the same orientation, but it's translated 2 meters into the Y-AXIS of the /turtle1 axis.

<p style="background:#3B8F10;color:white;" id="unit2_example4">
Python Program {U2.3.2.py}: moving_extra_frame_pub.py
</p>

In [ ]:
#!/usr/bin/env python  
import rospy
import tf
import math

if __name__ == '__main__':
    rospy.init_node('my_moving_carrot_tf_broadcaster')
    br = tf.TransformBroadcaster()
    rate = rospy.Rate(5.0)
    turning_speed_rate = 0.1
    while not rospy.is_shutdown():
        t = (rospy.Time.now().to_sec() * math.pi)*turning_speed_rate
        # Map to only one turn maximum [0,2*pi)
        rad_var = t % (2*math.pi)
        br.sendTransform((1.0 * math.sin(rad_var), 1.0 * math.cos(rad_var), 0.0),
                         (0.0, 0.0, 0.0, 1.0),
                         rospy.Time.now(),
                         "moving_carrot",
                         "turtle2")
        rate.sleep()

<p style="background:#3B8F10;color:white;">
END Python Program {U2.3.2.py}: moving_extra_frame_pub.py
</p>

The broadcaster publishes this new /moving_carrot frame referred to the /turtle2 frame. It has the same orientation, but it translates in a circular motion around the /turtle2 frame of the X-Y plane. The speed can be changed through the turning_speed_rate variable.

<p style="background:#EE9023;color:white;">Exercise 2.4</p>
<br>
In this exercise, you are going to make new versions of the two python examples <a href="#unit2_example3">{U2.3.1.py}: fixed_extra_frame_pub.py</a> and <a href="#unit2_example4">{U2.3.2.py}: moving_extra_frame_pub.py</a>.<br>
You will have to:
<ol>
    <li>Publish a fixed frame referred to the iRobot (/turtle2 frame) that is translated = (1.0,0,3.0).</li>
    <li>Publish a moving frame referred to the coke_can (/coke_can frame) that is translated (1.0,0,0) and turns on the Z-axis at a speed of 2 radians per second.</li>
</ol>

Things to bear in mind:
<ol>
    <li>You have to publish all the TF parent-frames that you use.</li>
    <li>When working with Quaternions, you will have to know how to go from Quaternions to Euler and back. This is because rotations in Euler are really intuitive, but not at all in Quaternions.</li>
</ol>

### Quaternion to Euler:

In [ ]:
import tf
quaternion = tf.transformations.quaternion_from_euler(roll,pitch,yaw)

### Euler to Quaternion

In [ ]:
import tf
quaternion_x = my_quaternion[0]
quaternion_y = my_quaternion[1]
quaternion_z = my_quaternion[2]
quaternion_w = my_quaternion[3]
explicit_quat = [quaternion_x, quaternion_y, quaternion_z, quaternion_w]
your_euler = tf.transformations.euler_from_quaternion(explicit_quat)

<p style="background:#EE9023;color:white;">END Exercise 2.4</p>

<p style="background:#EE9023;color:white;">Exercise 2.5</p>

Now, make your iRobot follow the new_frames (fixed_carrot and moving_carrot).

<p style="background:#EE9023;color:white;">END Exercise 2.5</p>